In [10]:
import numpy as np
import pandas as pd

# -----------------------------
# CONSTANTS (from manual)
# -----------------------------
g = 9.81                   # m/s^2
rho_oil = 929              # kg/m^3
rho_air = 1                # kg/m^3
eta = 1.8432e-5            # kg/m·s
C_corr = 6.17e-8           # m of Hg·m
P = 0.76                   # m of Hg
d = 5e-3                   # m (distance between plates)

# -----------------------------
# DERIVED CONSTANTS
# -----------------------------
C = 4 * np.pi * d * g * (rho_oil - rho_air) / 3
D = 9 * eta / (2 * g * (rho_oil - rho_air))
zeta = C_corr / (2 * P)

print(f"C = {C:.2f}")
print(f"D = {D:.3e}")
print(f"ζ = {zeta:.3e}")

# -----------------------------
# INPUT 
# -----------------------------

data_dynamic = pd.DataFrame({
    'Droplet': [1, 2, 3, 4, 5],
    'vf': [0.15e-3, 0.20e-3, 0.23e-3, 0.26e-3, 0.27e-3],  # m/s
    'tr': [2.0,4.2,1.8,2.2,3.7],  # arbitrary units, example
    'V': [326,326,348,383,361]  # volts
})

# -----------------------------
# DYNAMIC METHOD CALCULATIONS
# -----------------------------
data_dynamic['xi'] = D * data_dynamic['vf']
data_dynamic['r'] = (-zeta + np.sqrt(zeta**2 + data_dynamic['xi']))
data_dynamic['r3'] = data_dynamic['r']**3
data_dynamic['T'] = 1 + (data_dynamic['tr'].mean() / data_dynamic['tr'])
data_dynamic['ne'] = C * data_dynamic['T'] * data_dynamic['r3'] / data_dynamic['V']

# Normalize and compute e
ne_min = data_dynamic['ne'].min()
data_dynamic['ne/ne_min'] = data_dynamic['ne'] / ne_min
data_dynamic['n_eff'] = np.rint(data_dynamic['ne/ne_min'])
data_dynamic['e'] = data_dynamic['ne'] / data_dynamic['n_eff']

# Display results
print("\nDynamic Method Results:")
display(data_dynamic[['Droplet','xi','r','r3','T','ne','n_eff','e']])

# Compute mean and std
mean_e_dyn = data_dynamic['e'].mean()
std_e_dyn = data_dynamic['e'].std(ddof=1)
print(f"\nMean e (Dynamic): {mean_e_dyn:.2e} ± {std_e_dyn:.2e} C")

# -----------------------------
# BALANCING METHOD
# -----------------------------
data_bal = pd.DataFrame({
    'Droplet': [1, 2, 3, 4, 5],
    'vf': [0.41e-3, 0.43e-3, 0.5e-3, 0.27e-3, 0.43e-3],
    'Vb': [300,318,314,369,340]  # example balancing voltages
})

data_bal['xi'] = D * data_bal['vf']
data_bal['r'] = (-zeta + np.sqrt(zeta**2 + data_bal['xi']))
data_bal['r3'] = data_bal['r']**3
data_bal['ne'] = C * data_bal['r3'] / data_bal['Vb']

# Normalize and compute e
ne_min_bal = data_bal['ne'].min()
data_bal['ne/ne_min'] = data_bal['ne'] / ne_min_bal
data_bal['n_eff'] = np.rint(data_bal['ne/ne_min'])
data_bal['e'] = data_bal['ne'] / data_bal['n_eff']

print("\nBalancing Method Results:")
display(data_bal[['Droplet','xi','r','r3','ne','n_eff','e']])

mean_e_bal = data_bal['e'].mean()
std_e_bal = data_bal['e'].std(ddof=1)
print(f"\nMean e (Balancing): {mean_e_bal:.2e} ± {std_e_bal:.2e} C")


C = 190.67
D = 9.111e-09
ζ = 4.059e-08

Dynamic Method Results:


,Droplet,xi,r,r3,T,ne,n_eff,e
0,1,1.366656e-12,0.000001,1.439655e-18,2.390000,2.012400e-18,1.0,2.012400e-18
1,2,1.822208e-12,0.000001,2.247628e-18,1.661905,2.184683e-18,1.0,2.184683e-18
2,3,2.095539e-12,0.000001,2.788781e-18,2.544444,3.887798e-18,2.0,1.943899e-18
3,4,2.368871e-12,0.000001,3.368636e-18,2.263636,3.796100e-18,2.0,1.898050e-18
4,5,2.459981e-12,0.000002,3.570112e-18,1.751351,3.302354e-18,2.0,1.651177e-18



Mean e (Dynamic): 1.94e-18 ± 1.94e-19 C

Balancing Method Results:


,Droplet,xi,r,r3,ne,n_eff,e
0,1,3.735527e-12,0.000002,6.779011e-18,4.308446e-18,2.0,2.154223e-18
1,2,3.917748e-12,0.000002,7.291838e-18,4.372054e-18,2.0,2.186027e-18
2,3,4.555520e-12,0.000002,9.183956e-18,5.576680e-18,3.0,1.858893e-18
3,4,2.459981e-12,0.000002,3.570112e-18,1.844723e-18,1.0,1.844723e-18
4,5,3.917748e-12,0.000002,7.291838e-18,4.089156e-18,2.0,2.044578e-18



Mean e (Balancing): 2.02e-18 ± 1.60e-19 C
